In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
RAW_LIVE_TRAINS = "/Volumes/rail_lakehouse/bronze_rail/raw_files/live_trains"
CHECKPOINT_LIVE = "/Volumes/rail_lakehouse/bronze_rail/checkpoints/live_trains"

print(f"📂 Raw live trains: {RAW_LIVE_TRAINS}")
print(f"💾 Checkpoint: {CHECKPOINT_LIVE}")


In [0]:
df_live_raw = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", f"{CHECKPOINT_LIVE}/schema")
    .option("cloudFiles.schemaEvolutionMode", "rescue")
    .load(RAW_LIVE_TRAINS)
)

In [0]:
df_live_bronze = (
    df_live_raw
    .select(
        col("*"),
    )
    .withColumn("_source_file", col("_metadata.file_path"))
    .withColumn("ingest_ts", current_timestamp())
)

In [0]:
query_live_bronze = (
    df_live_bronze.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"{CHECKPOINT_LIVE}/bronze")
    .trigger(availableNow=True)
    .table("rail_lakehouse.bronze_rail.bronze_live_trains")
)

In [0]:
query_live_bronze.awaitTermination()

In [0]:
%sql
SELECT *
FROM rail_lakehouse.bronze_rail.bronze_live_trains 
LIMIT 5